In [14]:
import pandas as pd
# save filepath to variable for easier access
#pump_file_path = '../data/processed/pomp_dataset.csv'
pump_file_path = "../data/processed/pump_dataset_full_cleaned.csv"
# read the data and store data in DataFrame titled pomp_data
pump_data = pd.read_csv(pump_file_path) 
# print a summary of the data in water pomp data
pump_data.describe()

,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year,year_recorded,month_recorded,dayofweek_recorded,pump_age,location_cluster
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,35.136692,-5.706033e+00,15.297003,5.629747,179.909983,1996.254495,2011.921667,4.375640,2.939933,15.667172,3.761549
std,2997.574558,693.116350,2.628459,2.946019e+00,17.587406,9.633649,471.482176,10.143783,0.958758,3.029247,1.951627,10.157652,2.820036
min,0.000000,-90.000000,29.219817,-1.164944e+01,1.000000,0.000000,0.000000,1960.000000,2002.000000,1.000000,0.000000,-7.000000,0.000000
25%,0.000000,0.000000,33.249501,-8.540621e+00,5.000000,2.000000,0.000000,1993.000000,2011.000000,2.000000,1.000000,8.000000,1.000000
50%,0.000000,369.000000,35.000424,-5.021597e+00,12.000000,3.000000,25.000000,1996.000000,2012.000000,3.000000,3.000000,15.000000,3.000000
75%,20.000000,1319.250000,37.241238,-3.326156e+00,17.000000,5.000000,215.000000,2004.000000,2013.000000,7.000000,5.000000,19.000000,6.000000
max,350000.000000,2770.000000,40.391390,-2.000000e-08,99.000000,80.000000,30500.000000,2013.000000,2013.000000,12.000000,6.000000,53.000000,9.000000


In [15]:
# Select subset of predictors
#cols_to_use = ['basin', 'ward', 'extraction_type_class', 'longitude', 'latitude', 'population', 'gps_height']
cols_to_use = ['basin',
    'extraction_type_class',
    'longitude',
    'latitude',
    'population',
    'gps_height',
    'management',
    'payment_type',
    'water_quality',
    'source_class',
    'quantity', 
    'pump_age', 'location_cluster', 'funder', 'installer', 'ward']

X = pump_data[cols_to_use]

# select target
y = pump_data.status_group

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestClassifier(n_estimators=50,
                                                              random_state=0))
                             ])

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np


categorical_cols = ['basin', 'extraction_type_class', 'funder', 'installer', 'ward']
numerical_cols = ['longitude', 'latitude', 'population', 'gps_height', 'pump_age', 'location_cluster']

# Define transformers
numerical_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Pipeline
model = RandomForestClassifier(n_estimators=50, random_state=0)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)])

# Cross-validation
scores = cross_val_score(my_pipeline, X, y, cv=5, scoring='accuracy')
print("Accuracy scores:\n", scores)
print("Mean accuracy:", scores.mean())


Accuracy scores:
 [0.75597643 0.75210438 0.75277778 0.75151515 0.7537037 ]
Mean accuracy: 0.7532154882154882
